In [1]:
import pandas as pd

data_path = './data/car-sim1/'
img_path = './data/car-sim1/IMG/'
df = pd.read_csv(data_path+'driving_log.csv', header=None)

df.head()

,0,1,2,3,4,5,6
0,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
1,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000012
2,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
3,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000004
4,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000010


In [2]:
import csv
import cv2
import numpy as np

lines = []
with open(data_path+'driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    #next(reader, None)
    for line in reader:
        lines.append(line)
        
def process_image(img): 
    # preprocess input
    return img


In [3]:
images = []
measurements = []
for line in lines:
    '''
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = img_path + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    '''
    
    steering_center = float(line[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    filename = line[0].split('/')[-1]
    img_center = process_image(cv2.imread(img_path + filename))
    filename = line[1].split('/')[-1]
    img_left = process_image(cv2.imread(img_path + filename))
    filename = line[2].split('/')[-1]
    img_right = process_image(cv2.imread(img_path + filename))

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
X_train = np.array(images)
Y_train = np.array(measurements)

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (6861, 160, 320, 3)
Y_train: (6861,)


In [3]:
images_center = []
images_left = []
images_right = []
measurements_center = []
measurements_left = []
measurements_right = []
for line in lines:

    steering_center = float(line[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    filename = line[0].split('/')[-1]
    img_center = process_image(cv2.imread(img_path + filename))
    
    filename = line[1].split('/')[-1]
    img_left = process_image(cv2.imread(img_path + filename))
    
    filename = line[2].split('/')[-1]
    img_right = process_image(cv2.imread(img_path + filename))
    
    # add images and angles to data set
    images_center.append(img_center)
    images_left.append(img_left)
    images_right.append(img_right)
    
    measurements_center.append(steering_center)
    measurements_left.append(steering_left)
    measurements_right.append(steering_right)
    
# convert to array
images_center = np.array(images_center)
images_left = np.array(images_left)
images_right = np.array(images_right)

measurements_center = np.array(measurements_center)
measurements_left = np.array(measurements_left)
measurements_right = np.array(measurements_right)
    
print('images:', images_center.shape)
print('measurements:', measurements_center.shape)

images: (2287, 160, 320, 3)
measurements: (2287,)


## Split data and augment

In [4]:
# split 70/15/15
length = len(measurements_center)
train_size = int(length * 0.7)

indices = np.random.permutation(length)
train_idx, test_idx = indices[:train_size], indices[train_size:]

val_size = int(len(test_idx) * 0.5)
val_idx, test_idx = test_idx[:val_size], test_idx[val_size:]

print('train_idx:', len(train_idx))
print('val_idx:', len(val_idx))
print('test_idx:', len(test_idx))

images_train = np.concatenate((
    images_center[train_idx], 
    images_left[train_idx], 
    images_right[train_idx]
))
measurements_train = np.concatenate((
    measurements_center[train_idx], 
    measurements_left[train_idx], 
    measurements_right[train_idx]
))

images_val, measurements_val = images_center[val_idx], measurements_center[val_idx]
images_test, measurements_test = images_center[test_idx], measurements_center[test_idx]

# augmented train set
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images_train, measurements_train):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
images_train = np.array(augmented_images)
measurements_train = np.array(augmented_measurements)
    
    
print('images_train:', images_train.shape)
print('measurements_train:', measurements_train.shape)
print('images_val:', images_val.shape)
print('measurements_val:', measurements_val.shape)
print('images_test:', images_test.shape)
print('measurements_test:', measurements_test.shape)

train_idx: 1600
val_idx: 343
test_idx: 344
images_train: (9600, 160, 320, 3)
measurements_train: (9600,)
images_val: (343, 160, 320, 3)
measurements_val: (343,)
images_test: (344, 160, 320, 3)
measurements_test: (344,)


In [102]:
X_train = images_train
Y_train = measurements_train

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (9600, 160, 320, 3)
Y_train: (9600,)


## First Model

In [33]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.optimizers import Adam

model = Sequential()
#model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=2)

model.save('model.h5')

Train on 10972 samples, validate on 2744 samples
Epoch 1/2
10972/10972 [==============================] - 7s 677us/step - loss: 0.0980 - val_loss: 0.1220
Epoch 2/2
10972/10972 [==============================] - 7s 623us/step - loss: 0.0578 - val_loss: 0.1509


## Second Model

In [49]:
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten, Dense, Lambda, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save('model.h5')

Train on 10977 samples, validate on 2745 samples
Epoch 1/3
10977/10977 [==============================] - 10s 889us/step - loss: 0.5664 - val_loss: 0.0745
Epoch 2/3
10977/10977 [==============================] - 9s 796us/step - loss: 0.0437 - val_loss: 0.0645
Epoch 3/3
10977/10977 [==============================] - 9s 803us/step - loss: 0.0374 - val_loss: 0.0639


## Third Model

In [5]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from tqdm import tqdm

modelTransfer = ResNet50(weights='imagenet', include_top=False)

def get_bottleneck_feature(img):
    # convert 3D tensor to 4D tensor with shape (1, ?, ?, 3) and return 4D tensor
    x = np.expand_dims(img, axis=0)
    
    # preprocess input
    x = preprocess_input(x)
    
    return modelTransfer.predict(x)

def get_bottleneck_features(images):
    bottleneck_features = []

    for img in tqdm(images):
        bottleneck_features.append(get_bottleneck_feature(img))
        #try:
        #    bottleneck_features.append(bottleneck_feature(img))
        #except OSError:
        #    None 
            
    return np.vstack(bottleneck_features)

modelTransfer.summary()

/home/dra/anaconda3/envs/tensorflow3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/dra/anaconda3/envs/tensorflow3.6/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [114]:
import pickle

X_train = get_bottleneck_features(images_train)
Y_train = measurements_train

X_val = get_bottleneck_features(images_val)
Y_val = measurements_val

X_test = get_bottleneck_features(images_test)
Y_test = measurements_test

bottleneck_feature = {
    'X_train': X_train,
    'Y_train': Y_train,
    'X_val': X_val,
    'Y_val': Y_val,
    'X_test': X_test,
    'Y_test': Y_test
}

with open('models/bottleneck_feature.pickle', 'wb') as f:
    pickle.dump(bottleneck_feature, f, pickle.HIGHEST_PROTOCOL)
    
print('X_train:', X_train.shape)
print('X_val:', X_val.shape)
print('X_test:', X_test.shape)

100%|██████████| 344/344 [00:03<00:00, 89.41it/s]


X_train: (9600, 5, 10, 2048)
X_val: (343, 5, 10, 2048)
X_test: (344, 5, 10, 2048)


In [6]:
import pickle

with open('models/bottleneck_feature.pickle', 'rb') as f:
    bottleneck_feature = pickle.load(f)
    
X_train = bottleneck_feature['X_train']
Y_train = bottleneck_feature['Y_train']

X_val = bottleneck_feature['X_val']
Y_val = bottleneck_feature['Y_val']

X_test = bottleneck_feature['X_test']
Y_test = bottleneck_feature['Y_test']

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(9600, 5, 10, 2048)
(9600,)
(343, 5, 10, 2048)
(343,)
(344, 5, 10, 2048)
(344,)


In [16]:
from keras.layers import Dense, Flatten, BatchNormalization
from keras.models import Sequential
from keras.regularizers import l1, l2

model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.02)))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.02)))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 102400)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 102400)            409600    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               26214656  
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 26,690,305
Trainable params: 26,485,505
Non-trainable params: 204,800
_________________________________________________________________


In [17]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          shuffle=True, epochs=100, callbacks=[checkpointer], verbose=1)

model.save('models/model.h5')

Train on 9600 samples, validate on 343 samples
Epoch 1/100
9600/9600 [==============================] - 10s 1ms/step - loss: 82.2855 - val_loss: 46.2307

Epoch 00001: val_loss improved from inf to 46.23071, saving model to models/weights.best.hdf5
Epoch 2/100
9600/9600 [==============================] - 7s 771us/step - loss: 34.5996 - val_loss: 23.6734

Epoch 00002: val_loss improved from 46.23071 to 23.67339, saving model to models/weights.best.hdf5
Epoch 3/100
9600/9600 [==============================] - 8s 784us/step - loss: 17.0653 - val_loss: 11.6336

Epoch 00003: val_loss improved from 23.67339 to 11.63360, saving model to models/weights.best.hdf5
Epoch 4/100
9600/9600 [==============================] - 7s 774us/step - loss: 8.7181 - val_loss: 6.5378

Epoch 00004: val_loss improved from 11.63360 to 6.53779, saving model to models/weights.best.hdf5
Epoch 5/100
9600/9600 [==============================] - 7s 775us/step - loss: 5.4122 - val_loss: 4.6311

Epoch 00005: val_loss improv

9600/9600 [==============================] - 8s 787us/step - loss: 0.0828 - val_loss: 0.0921

Epoch 00044: val_loss did not improve from 0.08729
Epoch 45/100
9600/9600 [==============================] - 8s 791us/step - loss: 0.0833 - val_loss: 0.1038

Epoch 00045: val_loss did not improve from 0.08729
Epoch 46/100
9600/9600 [==============================] - 8s 797us/step - loss: 0.0829 - val_loss: 0.0949

Epoch 00046: val_loss did not improve from 0.08729
Epoch 47/100
9600/9600 [==============================] - 8s 799us/step - loss: 0.0836 - val_loss: 0.0937

Epoch 00047: val_loss did not improve from 0.08729
Epoch 48/100
9600/9600 [==============================] - 8s 790us/step - loss: 0.0766 - val_loss: 0.0843

Epoch 00048: val_loss improved from 0.08729 to 0.08426, saving model to models/weights.best.hdf5
Epoch 49/100
9600/9600 [==============================] - 8s 790us/step - loss: 0.0731 - val_loss: 0.0855

Epoch 00049: val_loss did not improve from 0.08426
Epoch 50/100
9600/9

9600/9600 [==============================] - 8s 802us/step - loss: 0.0462 - val_loss: 0.0585

Epoch 00093: val_loss did not improve from 0.05562
Epoch 94/100
9600/9600 [==============================] - 8s 795us/step - loss: 0.0470 - val_loss: 0.0570

Epoch 00094: val_loss did not improve from 0.05562
Epoch 95/100
9600/9600 [==============================] - 8s 791us/step - loss: 0.0465 - val_loss: 0.0601

Epoch 00095: val_loss did not improve from 0.05562
Epoch 96/100
9600/9600 [==============================] - 8s 791us/step - loss: 0.0460 - val_loss: 0.0608

Epoch 00096: val_loss did not improve from 0.05562
Epoch 97/100
9600/9600 [==============================] - 8s 786us/step - loss: 0.0462 - val_loss: 0.0578

Epoch 00097: val_loss did not improve from 0.05562
Epoch 98/100
9600/9600 [==============================] - 8s 782us/step - loss: 0.0460 - val_loss: 0.0572

Epoch 00098: val_loss did not improve from 0.05562
Epoch 99/100
9600/9600 [==============================] - 8s 799u

In [21]:
from keras.models import load_model

model = load_model('models/model.h5')
model.load_weights('models/weights.best.hdf5')
model.save('models/model.best.h5')

#model.evaluate(X_test, Y_test)